# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> implement any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`bucket()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1615868736410_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-3>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [46]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

from pyspark.sql.functions import col
from pyspark.sql import *

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-spring2021"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp,lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [47]:
def user():
    # Returns a string consisting of your GT username.
    return 'rsoliman3'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [48]:
def long_trips(trips):
    # Returns a Dataframe with Schema the same as :trips:
    filtered_trips = trips.filter(col("Trip_distance") >= 2)
    #filtered_trips.show()
    return filtered_trips
    pass



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [49]:
def manhattan_trips(trips, lookup):
    # Returns a Dataframe with Schema: DOLocationID, pcount
    #from pyspark.sql.types import IntegerType
    #from pyspark.sql.functions import sum
    
    lookup = lookup.filter(col("Borough") == "Manhattan")
    joined = trips.join(lookup, trips.DOLocationID == lookup.LocationID)\
    manhatten_pcount =  joined.groupBy("DOLocationID").agg({"passenger_count": "sum" })\
                        .withColumnRenamed("sum(passenger_count)", "pcount")\
                       .orderBy(col("pcount").desc()).limit(20)
    return manhatten_pcount
    pass

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [50]:
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    pu_agg_avg = trips.groupBy("PULocationID").agg({"total_amount":"avg"})\
                 .withColumnRenamed("avg(total_amount)","avg_total_amount")
                 
    pu_agg_count = trips.groupBy("PULocationID").agg({"*": "count"})\
                   .withColumnRenamed("count(1)","count_DO")\
                   .withColumnRenamed("PULocationID","PULocationID_cp1")
    
    ma_trips_join = mtrips.join(trips, mtrips.DOLocationID == trips.DOLocationID )
    PU_topDU_grouped = ma_trips_join.groupBy("PULocationID").agg({"*": "count"})\
                       .withColumnRenamed("count(1)","count_DO_To_Popular")\
                       .withColumnRenamed("PULocationID","PULocationID_cp2")
    
    all_joined = pu_agg_avg.join(pu_agg_count, pu_agg_avg.PULocationID == pu_agg_count.PULocationID_cp1).join(PU_topDU_grouped, pu_agg_count.PULocationID_cp1 == PU_topDU_grouped.PULocationID_cp2)\
                 .drop(col("PULocationID_cp1")).drop(col("PULocationID_cp2"))\
                 .withColumn("weighted_profit", (col("count_DO_To_Popular")/ col("count_DO"))*col("avg_total_amount"))\
                 .drop(col("count_DO_To_Popular")).drop(col("count_DO")).drop(col("avg_total_amount"))\
                 .orderBy(col("weighted_profit").desc())
    return all_joined
    pass

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [51]:
def final_output(calc, lookup): 
    # Returns a Dataframe with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    final = calc.join(lookup, calc.PULocationID == lookup.LocationID )\
            .drop(col("PULocationID")).drop(col("LocationID")).drop(col("service_zone"))\
            .orderBy(col("weighted_profit").desc()).limit(20)
    final_rearranged = final.select("Zone", "Borough", "weighted_profit")
    return final_rearranged
    pass

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-info">
    Test your code on the small dataset first, as the large dataset will take a significantly longer time to run
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> use the same bucket url for multiple runs of the `main()` function, as this will cause errors. Make sure to change the name of your output location every time. (ie: s3://cse6242-gburdell3/output-small2)
</div>

Update the below cell with the path to your bucket, then run the below cell to run your code to store the results in S3.

When you have confirmed the results of the small dataset, run it again using the large dataset. Your output file will appear in a folder in your s3 bucket called YOUROUTPUT.csv as a csv file with a name something like part-0000-4d992f7a-0ad3-48f8-8c72-0022984e4b50-c000.csv. Download this file and rename it to q3_output.csv for submission. Do not make any other changes to the file. 

In [56]:
# update your bucket path
#bucket = 's3://cse6242-rsoliman3/output-small4'
#main('small', bucket)

bucket = 's3://cse6242-rsoliman3/output-large'
main('large', bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rsoliman3
Trip Count:  187203269
+--------------------+-------------+------------------+
|                Zone|      Borough|   weighted_profit|
+--------------------+-------------+------------------+
|        Baisley Park|       Queens|29.360455779130838|
|Flushing Meadows-...|       Queens|27.304845733617668|
|       South Jamaica|       Queens| 26.29491623987347|
|     Randalls Island|    Manhattan|24.150989940227518|
|        Astoria Park|       Queens| 21.70641711214753|
|Briarwood/Jamaica...|       Queens| 19.94506463178934|
|Springfield Garde...|       Queens|19.468309288781914|
|             Jamaica|       Queens|  19.2839430001379|
|              Corona|       Queens| 18.22876924815598|
|   LaGuardia Airport|       Queens|   18.181338808373|
|         Jamaica Bay|       Queens|  17.1005294467579|
|             Maspeth|       Queens|17.005450640079545|
|Eltingville/Annad...|Staten Island|16.837764756944445|
|         JFK Airport|       Queens|16.777725348249636|
|        Batter

#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading
</div>

In [45]:
#(trips,lookup)=load_data()
#lookup.printSchema()
#f_trips = long_trips(trips)
#man = manhattan_trips(f_trips,lookup)
#man.show()
#wp= weighted_profit(f_trips, man)
#wp.show(wp.count())
#x = final_output(wp,lookup)
#x.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Trip Count:  7667792
root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)

+------------+------------------+
|PULocationID|   weighted_profit|
+------------+------------------+
|           6|31.784444444444446|
|          99|29.342499999999998|
|         187|24.357777777777777|
|          93| 24.33013773166695|
|          10|23.624694333498454|
|         194| 23.12457751514923|
|         215|22.174114939432236|
|         156| 21.78655692729767|
|         206| 21.49189349112426|
|           5|          20.29625|
|           8|19.965360000000004|
|         138|17.923153949604917|
|         171|17.585450424382717|
|         132|16.958978201177352|
|         130|14.879835188762476|
|         105|          14.79625|
|          58|14.613195266272191|
|         219|14.337175219423653|
|         157|13.940437160690957|
|          28|13.327861288088645|
|          59|13.209640831758